## Importing Libraries

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data pre processing

### Fetch data

In [82]:
data=pd.read_csv('/home/nailsonseat/Desktop/Datasets/dataset_sunny.csv')
data

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [83]:
data=data.to_numpy().T
data

array([['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy',
        'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast',
        'overcast', 'rainy'],
       ['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool', 'mild',
        'cool', 'mild', 'mild', 'mild', 'hot', 'mild'],
       ['high', 'high', 'high', 'high', 'normal', 'normal', 'normal',
        'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high'],
       [False, True, False, False, False, True, True, False, False,
        False, True, True, False, True],
       ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes',
        'yes', 'yes', 'yes', 'no']], dtype=object)

### Extract training and test data

In [84]:
print(data.shape)
a=np.array([1,2,3,4])

(5, 14)


In [85]:
def dataSplit(data,fraction):
    indices=np.arange(data.shape[1])
    np.random.shuffle(indices)
    data=data[:,indices]
    length=data.shape[1]-int(data.shape[1]*fraction)
    indices=np.arange(length)
    train,train_labels=data[:-1,:length],data[-1,:length]
    test,test_labels=data[:-1,length:],data[-1,length:]
    return train,train_labels,test,test_labels
    

In [86]:
X,Y,X_t,Y_t=dataSplit(data,0.2)
print(Y)
pd.DataFrame(X)


['yes' 'no' 'yes' 'yes' 'yes' 'yes' 'yes' 'no' 'no' 'yes' 'yes' 'no']


,0,1,2,3,4,5,6,7,8,9,10,11
0,rainy,rainy,rainy,overcast,overcast,overcast,rainy,rainy,sunny,sunny,sunny,sunny
1,mild,cool,mild,mild,hot,cool,cool,mild,hot,cool,mild,mild
2,normal,normal,high,high,high,normal,normal,high,high,normal,normal,high
3,False,True,False,True,False,True,False,True,False,False,True,False
